# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,darwin,-12.4611,130.8418,82.38,85,60,8.05,AU,1707243302
1,1,hermanus,-34.4187,19.2345,74.80,73,33,19.91,ZA,1707243438
2,2,mount pearl,47.5166,-52.7813,33.46,97,100,26.46,CA,1707243438
3,3,edinburgh of the seven seas,-37.0676,-12.3116,62.40,69,8,6.13,SH,1707243438
4,4,sao felix do xingu,-6.6447,-51.9950,88.81,59,100,7.23,BR,1707243321


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 800,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)
# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= 50) & (city_data_df["Cloudiness"]>=20) & (city_data_df["Wind Speed"] < 5) ]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,blackmans bay,-43.0167,147.3167,55.53,78,33,1.01,AU,1707243439
16,16,kongolo,-5.3833,27.0000,71.37,97,98,3.62,CD,1707243441
21,21,kununurra,-15.7667,128.7333,78.78,94,93,4.61,AU,1707243239
22,22,taro,-8.3862,115.2800,73.40,96,100,1.50,ID,1707243441
24,24,tura,25.5198,90.2201,57.79,87,52,3.67,IN,1707243442


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.iloc[:, [1, 8, 2, 3, 5]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
 
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,blackmans bay,AU,-43.0167,147.3167,78,
16,kongolo,CD,-5.3833,27.0000,97,
21,kununurra,AU,-15.7667,128.7333,94,
22,taro,ID,-8.3862,115.2800,96,
24,tura,IN,25.5198,90.2201,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {"categories":categories,
          "limit":limit,
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitute = row["Lng"]
    latitue = row["Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitute},{latitue},{radius}"
    params["bias"] = f"proximity:{longitute},{latitue}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
kongolo - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
taro - nearest hotel: Padma Resort Ubud
tura - nearest hotel: No hotel found
tuy hoa - nearest hotel: Vinh Tuan Hotel
puerto lopez - nearest hotel: Bucarica
santo antonio do ica - nearest hotel: No hotel found
palikir - national government center - nearest hotel: Mangrove Bay Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
mhamid - nearest hotel: Hotel Kasbah Azalay
yilan - nearest hotel: 蘭城晶英酒店(6-11F)
imbert - nearest hotel: No hotel found
tarawa - nearest hotel: Betio Lodge
holualoa - nearest hotel: Kona Hotel
mareeba - nearest hotel: The Gateway Hotel
sananduva - nearest hotel: No hotel found
kolwezi - nearest hotel: Guest House le Manguier
alofi - nearest hotel: Matavai Resort
chulucanas - nearest hotel: Hostal Vicús
georgetown - nearest hotel: Page 63 hostel
paragominas - nearest hotel: No hotel found
pimenta bueno - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
6,blackmans bay,AU,-43.0167,147.3167,78,Villa Howden
16,kongolo,CD,-5.3833,27.0000,97,No hotel found
21,kununurra,AU,-15.7667,128.7333,94,No hotel found
22,taro,ID,-8.3862,115.2800,96,Padma Resort Ubud
24,tura,IN,25.5198,90.2201,87,No hotel found
...,...,...,...,...,...,...
538,tome acu,BR,-2.4189,-48.1522,61,No hotel found
539,palafrugell,ES,41.9174,3.1631,52,Hostal Plaja
542,moyo,UG,3.6609,31.7247,35,Universal Lodge Moyo
544,parapat,ID,2.6630,98.9349,91,Hotel Danau Toba International


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = 0.5
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)